In [26]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm 

class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.classes = os.listdir(data_dir)  # training directories
        self.images = []  
        
        # looping class directories
        for class_dir in self.classes:
            class_path = os.path.join(data_dir, class_dir)
            if os.path.isdir(class_path):
                # image files in class directory
                image_files = [file for file in os.listdir(class_path) if file.endswith(('.jpg', '.jpeg', '.png'))]
                # Append image paths and labels to the images list
                self.images.extend([(os.path.join(class_path, image_file), class_dir) for image_file in image_files])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path, label = self.images[idx]
        try:
            # Load image data from the file
            with open(image_path, 'rb') as file:  # Open in binary mode
                image = Image.open(file)  # Open the image
                # Convert grayscale images to RGB format if needed
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                # resize and convertin to tensor
                transform = transforms.Compose([
                    transforms.Resize((256, 256)),  # Resize to a fixed size
                    transforms.ToTensor(),  # Convert to PyTorch tensor
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize 
                ])
                tensor_image = transform(image)
                return tensor_image, label
        except Exception as e:
            print(f"Error loading data from {image_path}: {e}")
            return None, None

data_path = "train" # Update directory path - preferrably put the code with the data

# instance of custom dataset
dataset = MyDataset(data_path)

# parameters for DataLoader
batch_size = 64
shuffle = True
num_workers = 0  # Adjust this based on your system's capabilities, 4 if system is good

# DataLoader instance
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

# Iterate over batches of data
for images, labels in tqdm(dataloader, desc="Training", unit="batch"):
    # Process your image data batch here
    pass

Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [03:03<00:00,  2.23s/batch]


In [ ]:
import torch.nn as nn
import torch.optim as optim

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  # image dimensions
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)  # reshaping img dimensions
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Initialize the model
model = CNNModel(num_classes=len(dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, torch.tensor([dataset.classes.index(label) for label in labels]))  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

print("Training finished.")

Epoch 1/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [12:24<00:00,  9.08s/batch]


Epoch 1/5, Loss: 0.6383


Epoch 2/5:  80%|██████████████████████████████████████████████████████████████████████████████████████████████████████▏                        | 66/82 [08:40<01:50,  6.89s/batch]